In [2]:
import os
import pandas as pd
import numpy as np
from os import listdir

In [3]:
from tensorflow.keras.layers import Concatenate, AveragePooling2D, BatchNormalization, Conv2D, Conv2DTranspose, Dense, Input, Reshape, Flatten
from tensorflow.keras import models
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras import backend as K

In [4]:
Xs = np.load("../data/Bank/Bank_Mean_x_denoise_600_1200rpm_Multilabel.npy")
ys = np.load("../data/Bank/Bank_Mean_y_denoise_600_1200rpm_Multilabel.npy")
Xs.shape, ys.shape
# ((58000, 6000, 3), (58000, 3))

((58000, 6000, 3), (58000, 3))

In [5]:
from tensorflow import keras
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow.keras.backend as K

def masking_loss(y_true,y_pred):
    diff=tf.cast(y_true, tf.float32)-y_pred
    mask=tf.cast((tf.abs(diff)>=0.5),tf.float32)
    loss=keras.losses.mse(mask*tf.cast(y_true, tf.float32), mask*y_pred)
    return loss

In [69]:
'''
BEST : Models/Raw_l12_masking_elu_Conv1D_MultiClass.h5
'''
from tensorflow.keras import regularizers
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv1D, Input, BatchNormalization, AveragePooling1D, Flatten, MaxPooling1D, Dropout, Dense

input = Input((Xs.shape[1], Xs.shape[2]))
x = Conv1D(8, 600, activation='relu', padding='same' , kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4))(input)
#x = Conv1D(32, 64, activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = AveragePooling1D(pool_size=4)(x)
# print(x.get_shape())
x = Conv1D(16, 300, activation='relu', padding='same')(x)
#x = Conv1D(64, 8, activation='relu', padding='same')(x)
#x = AveragePooling1D()(x)
x = AveragePooling1D(pool_size=4)(x)

x = Conv1D(32, 100, activation='relu', padding='same')(x)
#x = Conv1D(64, 3, activation='relu', padding='same')(x)
#x = AveragePooling1D()(x)
x = AveragePooling1D(pool_size=4)(x)
# print(x.get_shape())
x = Conv1D(64, 32, activation='relu', padding='same')(x)
#x = Conv1D(128, 3, activation='relu', padding='same')(x)
x = AveragePooling1D(pool_size=4)(x)

x = Conv1D(128, 8, activation='relu', padding='same')(x)
x = Conv1D(128, 8, activation='relu')(x)
x = AveragePooling1D(pool_size=2)(x)

x = Conv1D(256, 4, activation='relu', padding='same')(x)
x = Conv1D(256, 4, activation='relu', padding='same')(x)
x = AveragePooling1D(pool_size=2)(x)

x = Flatten()(x)
x = Dense(128, activation='relu' , kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4))(x)
x = Dropout(0.25)(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.25)(x)
x = Dense(3, activation='elu')(x)
model = Model(input, x)
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 6000, 3)]         0         
                                                                 
 conv1d_22 (Conv1D)          (None, 6000, 8)           14408     
                                                                 
 batch_normalization_3 (Batc  (None, 6000, 8)          32        
 hNormalization)                                                 
                                                                 
 average_pooling1d_17 (Avera  (None, 1500, 8)          0         
 gePooling1D)                                                    
                                                                 
 conv1d_23 (Conv1D)          (None, 1500, 16)          38416     
                                                                 
 average_pooling1d_18 (Avera  (None, 375, 16)          0   

In [6]:
'''
../Models/Raw_MaxPool_MoreDense_l12_masking_elu_Conv1D_MultiClass.h5
'''
from tensorflow.keras import regularizers
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv1D, Input, BatchNormalization, AveragePooling1D, Flatten, MaxPooling1D, Dropout, Dense

def reparameterize(mean, logvar):
    eps = tf.random.normal(shape=[128])
    return eps * tf.exp(logvar * .5) + mean

input = Input((Xs.shape[1], Xs.shape[2]))
x = Conv1D(8, 600, activation='relu', padding='same' , kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4))(input)
#x = Conv1D(32, 64, activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling1D (pool_size=4)(x)
# print(x.get_shape())
x = Conv1D(16, 300, activation='relu', padding='same', kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4))(x)
#x = Conv1D(64, 8, activation='relu', padding='same')(x)
#x = AveragePooling1D()(x)
x = AveragePooling1D(pool_size=4)(x)

x = Conv1D(32, 100, activation='relu', padding='same', kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4))(x)
#x = Conv1D(64, 3, activation='relu', padding='same')(x)
#x = AveragePooling1D()(x)
x = AveragePooling1D(pool_size=4)(x)
# print(x.get_shape())
x = Conv1D(64, 32, activation='relu', padding='same', kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4))(x)
#x = Conv1D(128, 3, activation='relu', padding='same')(x)
x = AveragePooling1D(pool_size=4)(x)

x = Conv1D(128, 8, activation='relu', padding='same', kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4))(x)
x = Conv1D(128, 8, activation='relu', padding='same')(x)
x = AveragePooling1D(pool_size=2)(x)

x = Conv1D(256, 4, activation='relu', padding='same', kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4))(x)
x = Conv1D(256, 4, activation='relu', padding='same')(x)
x = MaxPooling1D(pool_size=2)(x)

x = Flatten()(x)
x = Dense(256, activation='relu' , kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4))(x)
x = Dropout(0.25)(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.25)(x)
x = Dense(3, activation='elu')(x)
model = Model(input, x)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 6000, 3)]         0         
                                                                 
 conv1d (Conv1D)             (None, 6000, 8)           14408     
                                                                 
 batch_normalization (BatchN  (None, 6000, 8)          32        
 ormalization)                                                   
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1500, 8)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 1500, 16)          38416     
                                                                 
 average_pooling1d (AverageP  (None, 375, 16)          0     

2022-04-08 10:31:03.522479: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-08 10:31:04.107565: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13534 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:21:01.0, compute capability: 7.5


In [7]:
checkpointer = ModelCheckpoint(monitor='val_loss', filepath='../Models/Raw_MaxPool_MoreDense_l12_masking_elu_Conv1D_MultiClass.h5', 
                                       verbose=1, save_best_only= True, 
                                       mode = 'min',
                                       save_weight_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=10, min_lr=0.00001)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, 
                           patience=25)
callbacks = [checkpointer, reduce_lr, es]

In [8]:
model.compile(optimizer='adam', loss=[masking_loss], metrics=['accuracy'])

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(Xs, ys, test_size=0.2, random_state=42)
X_train.shape, X_test.shape

((46400, 6000, 3), (11600, 6000, 3))

In [ ]:
model.fit(X_train, y_train, batch_size=128,epochs=200 , validation_data=(X_test, y_test), callbacks=callbacks)

In [ ]:
## Evaluate

In [51]:
!pwd

/root/Notebook


In [32]:
from tensorflow import keras
# model = keras.models.load_model('../Models/triplet_elu_Conv1D_MultiClass.h5')
# Loads the weights
model.load_weights('../Models/Raw_MaxPool_MoreDense_l12_masking_elu_Conv1D_MultiClass.h5')

In [33]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 6000, 3)]         0         
                                                                 
 conv1d (Conv1D)             (None, 6000, 8)           14408     
                                                                 
 batch_normalization (BatchN  (None, 6000, 8)          32        
 ormalization)                                                   
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1500, 8)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 1500, 16)          38416     
                                                                 
 average_pooling1d (AverageP  (None, 375, 16)          0     

In [18]:
pred_val = model.predict(X_test)

In [19]:
pred_val

array([[-0.03959975, -0.10166721,  3.051476  ],
       [ 1.6467396 ,  1.7537956 , -0.05787564],
       [ 1.4909191 ,  1.846158  , -0.18499584],
       ...,
       [ 0.07819688,  2.9179587 , -0.17407507],
       [ 1.4740404 ,  1.8313091 , -0.17540298],
       [ 0.6353797 ,  0.03899473,  2.9903135 ]], dtype=float32)

In [34]:
y_val_around1 = np.clip(np.around(pred_val)[:, 0], 0, 3)
y_val_around1

y_val_around2 = np.clip(np.around(pred_val)[:, 1], 0, 3)
y_val_around2

y_val_around3 = np.clip(np.around(pred_val)[:, 2], 0, 4)
y_val_around3

array([0., 0., 3., ..., 0., 0., 1.], dtype=float32)

In [37]:
y_pred_val = []
for i in range(len(pred_val)):
  y1, y2, y3 = "", "", ""
  y1 = y_val_around1[i]
  y2 = y_val_around2[i]
  y3 = y_val_around3[i]
  y_pred_val.append(inverse_multilabeling(y1, y2, y3))

Found 29
Found 29
Found 29
Found 29
Found 29
Found 29
Found 29
Found 29
Found 29
Found 29
Found 29
Found 29
Found 29
Found 29
Found 29
Found 29
Found 29
Found 29
Found 29
Found 29
Found 29
Found 29
Found 29
Found 29
Found 29
Found 29
Found 29
Found 29
y1 : 0, y2 : 1, y3 : 1
Found 29
Found 29
Found 29
Found 29
Found 29
Found 29
Found 29
Found 29
Found 29
Found 29


In [38]:
ymax_test = []
for i in range(len(y_test)):
  y1, y2, y3 = "", "", ""
  y1 = y_test[i][0]
  y2 = y_test[i][1]
  y3 = y_test[i][2]
  ymax_test.append(inverse_multilabeling(y1, y2, y3))

In [39]:
from sklearn.metrics import confusion_matrix, classification_report

y_pred = y_pred_val
y_true = ymax_test
confusion_array = confusion_matrix(y_true, y_pred)
confusion_array

array([[374,  26,   3,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0],
       [ 24, 277,  72,  18,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0],
       [  7,  44, 299,  18,   0,   0,   0,   0,   0,   0,   0,   0,   1,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0],
       [  1,  34, 125, 263,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0],
       [  3,   4,   4,   0, 252,   0,   0,   0,   0,   0,   0,  97,  30,
          4,   0,   1,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0],
       [  0,   0,   0,   0,   0, 235,  16,   0,   0,   0,   0,   0,   0,
          0,  35,   4,   1,  39,  33,  11,   0,   0,

In [40]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.91      0.93      0.92       403
           1       0.72      0.71      0.71       391
           2       0.59      0.81      0.68       369
           3       0.88      0.62      0.73       423
           4       0.90      0.64      0.75       395
           5       0.71      0.62      0.67       376
           6       0.83      0.53      0.65       396
           7       0.99      0.45      0.62       412
           8       0.99      0.50      0.67       393
           9       0.94      0.97      0.96       411
          10       0.98      0.87      0.92       396
          11       0.65      0.80      0.72       422
          12       0.60      0.70      0.64       427
          13       0.76      0.67      0.71       407
          14       0.53      0.66      0.59       397
          15       0.59      0.68      0.63       415
          16       0.85      0.84      0.84       414
          17       0.74    

/root/miniconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/root/miniconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/root/miniconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [41]:
### Predict

In [20]:
Xs = np.load('../data/Mean_final_x_denoise.npy')
Xs.shape

(1800, 6000, 3)

In [34]:
ypred = model.predict(Xs)

In [35]:
ypred.shape

(1800, 3)

In [36]:
ypred[0:20]

array([[ 7.34559298e-02, -1.09300815e-01, -2.29601651e-01],
       [ 2.45327139e+00,  1.22193944e+00, -8.70240480e-02],
       [ 2.97339249e+00,  1.03491092e+00, -1.36759907e-01],
       [ 1.10281241e+00,  1.94137812e-01, -2.35696579e-03],
       [ 1.54996908e+00, -1.56156197e-01,  2.91342974e+00],
       [ 1.58916807e+00, -1.42560601e-01,  2.36105561e+00],
       [ 1.24422312e-01, -8.62440169e-02,  3.05472803e+00],
       [ 9.73662496e-01,  1.49765134e-01,  1.23056924e+00],
       [ 1.94930315e-01,  4.69213724e-02,  1.28514004e+00],
       [ 3.94209146e-01,  3.27546597e-02,  1.44198513e+00],
       [ 1.52122760e+00,  2.91417837e-02,  1.49340796e+00],
       [ 5.29798269e-02, -1.31286740e-01,  2.93860316e+00],
       [ 1.10281241e+00,  1.94137812e-01, -2.35696579e-03],
       [-5.96833182e-03, -1.34313866e-01,  2.57123566e+00],
       [ 2.42493582e+00,  1.50094056e+00, -1.24346316e-01],
       [ 1.55945170e+00, -1.61907092e-01,  6.30170286e-01],
       [-1.06112495e-01, -1.20516494e-01

In [37]:
y_pred_around1 = np.clip(np.around(ypred)[:, 0], 0, 3)
y_pred_around1

y_pred_around2 = np.clip(np.around(ypred)[:, 1], 0, 3)
y_pred_around2

y_pred_around3 = np.clip(np.around(ypred)[:, 2], 0, 4)
y_pred_around3

array([0., 0., 0., ..., 0., 2., 2.], dtype=float32)

In [38]:
def inverse_multilabeling(y1, y2, y3):
  y1 = int(y1)
  y2 = int(y2)
  y3 = int(y3)
  if(y1 == 0 and y2 == 0 and y3 == 0):
    return 0
  if(y1 == 1 and y2 == 0 and y3 == 0):
    return 1
  if(y1 == 2 and y2 == 0 and y3 == 0):
    return 2
  if(y1 == 3 and y2 == 0 and y3 == 0):
    return 3
  if(y1 == 0 and y2 == 1 and y3 == 0):
    return 4
  if(y1 == 0 and y2 == 2 and y3 == 0):
    return 5
  if(y1 == 0 and y2 == 3 and y3 == 0):
    return 6
  if(y1 == 0 and y2 == 0 and y3 == 1):
    return 7
  if(y1 == 0 and y2 == 0 and y3 == 2):
    return 8
  if(y1 == 0 and y2 == 0 and y3 == 3):
    return 9
  if(y1 == 0 and y2 == 0 and y3 == 4):
    return 10
  if(y1 == 1 and y2 == 1 and y3 == 0):
    return 11
  if(y1 == 2 and y2 == 1 and y3 == 0):
    return 12
  if(y1 == 3 and y2 == 1 and y3 == 0):
    return 13
  if(y1 == 1 and y2 == 2 and y3 == 0):
    return 14
  if(y1 == 2 and y2 == 2 and y3 == 0):
    return 15
  if(y1 == 3 and y2 == 2 and y3 == 0):
    return 16
  if(y1 == 1 and y2 == 3 and y3 == 0):
    return 17
  if(y1 == 2 and y2 == 3 and y3 == 0):
    return 18
  if(y1 == 3 and y2 == 3 and y3 == 0):
    return 19
  if(y1 == 1 and y2 == 0 and y3 == 1):
    return 20
  if(y1 == 2 and y2 == 0 and y3 == 1):
    return 21
  if(y1 == 3 and y2 == 0 and y3 == 1):
    return 22
  if(y1 == 1 and y2 == 0 and y3 == 2):
    return 23
  if(y1 == 2 and y2 == 0 and y3 == 2):
    return 24
  if(y1 == 3 and y2 == 0 and y3 == 2):
    return 25
  if(y1 == 1 and y2 == 0 and y3 == 3):
    return 26
  if(y1 == 2 and y2 == 0 and y3 == 3):
    return 27
  if(y1 == 3 and y2 == 0 and y3 == 3):
    return 28
  if(y1 != 0 and y2 != 0 and y3 != 0):
    print("Found 29")
    return 29
  print("y1 : {}, y2 : {}, y3 : {}".format(y1, y2, y3))
  return 0


In [ ]:
ymax = []
for i in range(len(y_pred_around1)):
  y1, y2, y3 = "", "", ""
  y1 = y_pred_around1[i]
  y2 = y_pred_around2[i]
  y3 = y_pred_around3[i]
  ymax.append(inverse_multilabeling(y1, y2, y3))

In [40]:
ids = []
predicts = []
for i in range(len(ymax)):
    predicts.append(ymax[i])
    ids.append("test_data{}.csv".format(i))

len(ids),len(predicts)

(1800, 1800)

In [41]:
import pandas as pd

df = pd.DataFrame(data={'filename':ids,'code': predicts})
df

,filename,code
0,test_data0.csv,0
1,test_data1.csv,12
2,test_data2.csv,13
3,test_data3.csv,1
4,test_data4.csv,27
...,...,...
1795,test_data1795.csv,0
1796,test_data1796.csv,11
1797,test_data1797.csv,6
1798,test_data1798.csv,23


In [42]:
df.code.value_counts()

9     165
27    147
23    112
26    111
0     109
7     100
12     95
24     94
1      73
4      73
8      70
2      60
20     56
15     52
21     52
17     50
14     42
11     40
18     39
5      37
29     34
6      34
28     25
25     25
10     24
13     19
3      18
16     17
22     16
19     11
Name: code, dtype: int64

In [30]:
df.to_csv('MoreDense_Raw_L12_FinalTest_elu_Conv1D_MultiClass.csv', index=False)

In [73]:
idx = df[df['code']==29].index
idx

Int64Index([ 159,  205,  231,  369,  423,  446,  736,  860,  897,  957,  970,
            1205, 1249, 1518, 1736, 1763],
           dtype='int64')

In [70]:
Non_df = pd.read_csv('../data/Non/CNN_denoise_1.csv')
Non_df

,filename,code
0,test_data0.csv,0
1,test_data1.csv,18
2,test_data2.csv,13
3,test_data3.csv,1
4,test_data4.csv,27
...,...,...
1795,test_data1795.csv,0
1796,test_data1796.csv,9
1797,test_data1797.csv,11
1798,test_data1798.csv,23


In [74]:
Non_df['code'][idx] = 29

/tmp/ipykernel_18487/2976816564.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bank_df['code'][idx] = 29


In [75]:
Non_df[Non_df['code']==29].index

Int64Index([ 159,  205,  231,  369,  423,  446,  736,  860,  897,  957,  970,
            1205, 1249, 1518, 1736, 1763],
           dtype='int64')

In [78]:
Non_df.to_csv("Non_All_class_Multilabel.csv", index = False)